In [1]:
# 05_Kriging_Interpolation_LocalGuideline_withRoads.py
# Outputs per day:
#   (A) Continuous heatmap (RdBu_r), fixed 0–100 scale + guideline contours → OUT_F_CONT
#   (B) Categorical map using local PM2.5 categories (green→maroon) → OUT_F_CAT
# Also writes a GeoTIFF with the kriged grid per day → OUT_R_DIR
# Adds Pasig road network overlay (primary/secondary/tertiary); no basemap.

import os, glob, warnings, re
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import rasterio
from rasterio.transform import from_origin
from shapely.ops import unary_union
from shapely.geometry import Point
from fiona.env import Env
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# -------- Paths (UPDATED)
DATA_DIR    = r"C:\Users\HP\Documents\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP   = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
ROADS_SHP   = r"C:\Users\HP\Documents\PhD Class\Shapefile\MM\Pasig\PasigRN.shp"
OUT_R_DIR   = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\rasters"
OUT_F_CONT  = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local\continuous"
OUT_F_CAT   = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local\categorical"
os.makedirs(OUT_R_DIR, exist_ok=True)
os.makedirs(OUT_F_CONT, exist_ok=True)
os.makedirs(OUT_F_CAT, exist_ok=True)

# -------- CRS & figure
WGS84  = "+proj=longlat +datum=WGS84 +no_defs"
UTM51  = "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7, 7), 150

# -------- Grid/Kriging
GRID_RES_M = 100
VARIOGRAM_MODEL = "spherical"
RETRY_JITTER = 1e-3           # tiny noise (µg/m³) for retry if matrix is singular
COVERAGE_MAX_DIST_M = 3000

# -------- Continuous (analysis) style — red high, blue low
CMAP_CONT = "RdBu_r"
VMIN, VMAX = 0.0, 100.0
NORM_CONT = mcolors.PowerNorm(gamma=0.6, vmin=VMIN, vmax=VMAX)
GUIDE_LEVELS = [25, 35, 45, 55, 90]

# -------- Local guideline bins + colors (categorical/policy)
BINS    = [0.0, 25.0, 35.0, 45.0, 55.0, 90.0, np.inf]
LABELS  = [
    "Good (0–25.0)",
    "Fair (25.1–35.0)",
    "Unhealthy (sensitive) (35.1–45.0)",
    "Very unhealthy (45.1–55.0)",
    "Acutely unhealthy (55.1–90.0)",
    "Emergency (≥91)"
]
COLORS  = ["#00E400", "#FFFF00", "#FF7E00", "#FF0000", "#8F3F97", "#7E0023"]
CMAP_CAT = mpl.colors.ListedColormap(COLORS)
NORM_CAT = mpl.colors.BoundaryNorm(BINS, ncolors=len(COLORS), clip=False)

def pick_pm_col(cols):
    for c in cols:
        if str(c).lower() in ("pm25", "pm_25", "pm2_5", "pm2.5"):
            return c
    return None

# -------- Exclusion (for kriging only)
EXCLUDE_FOR_KRIGING = True
FLAG_MICRO_SITES = {
    "san antonio barangay hall",
    "san antonio brgy hall",
    "san antonio brgy. hall",
}
def _norm_name(s):
    return re.sub(r"\s+", " ", str(s).strip().lower())

# Optional coordinate-based buffer (WGS84 -> UTM51); set None if unused
BBQ_COORD_WGS84 = None
BBQ_RADIUS_M    = 25.0

# -------- Boundary & grid
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None:
    pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

minx, miny, maxx, maxy = geom.bounds
xs = np.arange(minx, maxx + GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy + GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)
transform = from_origin(xs.min(), ys.max(), GRID_RES_M, GRID_RES_M)

# -------- Roads (clip to Pasig; keep only primary/secondary/tertiary)
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)
roads_utm = roads.to_crs(UTM51)

# choose best column
road_col = None
for c in ("highway","fclass","type"):
    if c in roads_utm.columns:
        road_col = c; break
if road_col is None:
    raise SystemExit("No road class column ('highway'/'fclass'/'type') found in road shapefile.")
roads_utm[road_col] = roads_utm[road_col].astype(str).str.lower()
wanted = {"primary","secondary","tertiary"}
roads_sel = roads_utm[roads_utm[road_col].isin(wanted)].copy()
try:
    roads_clip = gpd.clip(roads_sel, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel, pasig_utm[["geometry"]], how="intersection")

# style + legend handles
STYLE_MAP = {
    "primary":   {"linewidth": 2.0, "color": "#d62728"},
    "secondary": {"linewidth": 1.6, "color": "#1f77b4"},
    "tertiary":  {"linewidth": 1.2, "color": "#2ca02c"},
}
ROAD_HANDLES = [
    Line2D([0],[0], color=STYLE_MAP["primary"]["color"],   lw=STYLE_MAP["primary"]["linewidth"],   label="Primary"),
    Line2D([0],[0], color=STYLE_MAP["secondary"]["color"], lw=STYLE_MAP["secondary"]["linewidth"], label="Secondary"),
    Line2D([0],[0], color=STYLE_MAP["tertiary"]["color"],  lw=STYLE_MAP["tertiary"]["linewidth"],  label="Tertiary"),
]

# BBQ buffer (if coordinate provided)
if BBQ_COORD_WGS84 is not None:
    bbq_pt_utm = gpd.GeoSeries([Point(BBQ_COORD_WGS84)], crs=WGS84).to_crs(UTM51).iloc[0]
    BBQ_BUFFER = gpd.GeoSeries([bbq_pt_utm.buffer(BBQ_RADIUS_M)], crs=UTM51)
else:
    BBQ_BUFFER = None

# -------- Iterate daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files:
    raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    day  = base.replace("date_", "")

    g = gpd.read_file(f)
    if g.crs is None:
        g = g.set_crs(WGS84)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude", "lon", "x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude", "lat", "y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print("Skip (no geometry):", base)
            continue

    pm_col = pick_pm_col(g.columns)
    if pm_col is None:
        print("Skip (no PM2.5):", base)
        continue

    # Points in UTM, de-duplicate exact XY, clip negatives
    pts = g.to_crs(UTM51)[[pm_col, "geometry"]].copy()
    pts["x"] = pts.geometry.x
    pts["y"] = pts.geometry.y
    pts = pts.groupby(["x", "y"], as_index=False)[pm_col].mean()
    pts = gpd.GeoDataFrame(pts, geometry=gpd.points_from_xy(pts["x"], pts["y"], crs=UTM51))

    # ---- Exclusion for kriging
    if EXCLUDE_FOR_KRIGING:
        st_col = next((c for c in ("stations","station","Station","STATION") if c in g.columns), None)
        if st_col is not None:
            g_tmp = g.to_crs(UTM51)[[st_col, "geometry"]].copy()
            g_tmp["station_norm"] = g_tmp[st_col].astype(str).map(_norm_name)
            pts = gpd.sjoin(pts, g_tmp[["station_norm","geometry"]], how="left", predicate="intersects")
            pts["is_flagged"] = pts["station_norm"].isin(FLAG_MICRO_SITES)
        else:
            pts["is_flagged"] = False
        if BBQ_BUFFER is not None:
            within_buf = pts.geometry.within(BBQ_BUFFER.iloc[0])
            pts.loc[within_buf, "is_flagged"] = True
        pts_krig = pts[~pts["is_flagged"]].copy()
    else:
        pts_krig = pts.copy()

    if pts_krig.empty or pts_krig.shape[0] < 3:
        print("Skip (n<3 after exclusion):", base)
        continue

    x = pts_krig["x"].to_numpy()
    y = pts_krig["y"].to_numpy()
    z = np.clip(pts_krig[pm_col].astype(float).to_numpy(), 0, None)

    # --- Kriging with retry using jitter
    try:
        OK = OrdinaryKriging(x, y, z,
                             variogram_model=VARIOGRAM_MODEL,
                             enable_plotting=False, verbose=False)
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zg = np.array(zg)
    except LinAlgError:
        z_jitter = z + np.random.normal(0, RETRY_JITTER, size=z.shape)
        OK = OrdinaryKriging(x, y, z_jitter,
                             variogram_model=VARIOGRAM_MODEL,
                             enable_plotting=False, verbose=False)
        zg, zv = OK.execute("grid", xs, ys, backend="loop")
        zg = np.array(zg)

    # --- Coverage + boundary masks
    tree = cKDTree(np.c_[x, y])
    dists, _ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
    cover = (dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M)
    centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
    inside = centers.within(unary_union(pasig_utm.geometry)).to_numpy().reshape(zg.shape)
    valid = cover & inside
    zmask = np.where(valid, np.maximum(zg, 0.0), np.nan).astype("float32")
    if not np.isfinite(zmask).any():
        print("Skip (no valid cells):", base)
        continue

    # --- Save GeoTIFF
    tif = os.path.join(OUT_R_DIR, f"{base}_pm25_kriged.tif")
    with rasterio.open(tif, "w", driver="GTiff",
                       height=zmask.shape[0], width=zmask.shape[1],
                       count=1, dtype="float32", crs=UTM51, transform=transform,
                       nodata=np.nan) as dst:
        dst.write(zmask, 1)

    # === (A) Continuous map + roads
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    ax.imshow(zmask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
              origin="lower", cmap=CMAP_CONT, norm=NORM_CONT)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)

    # roads overlay
    for klass, style in STYLE_MAP.items():
        sub = roads_clip.loc[roads_clip[road_col] == klass]
        if not sub.empty:
            sub.plot(ax=ax, **style, zorder=4)

    # stations used for kriging
    s = 30 + 170 * (z / max(1e-6, z.max()))
    ax.scatter(x, y, c=z, s=s, cmap=CMAP_CONT, norm=NORM_CONT,
               edgecolors="white", linewidths=0.8, zorder=5)

    # flagged points (excluded) for transparency
    if EXCLUDE_FOR_KRIGING and "is_flagged" in pts.columns and pts["is_flagged"].any():
        flagged = pts[pts["is_flagged"]]
        ax.scatter(flagged["x"], flagged["y"], marker="X", s=110, color="none",
                   edgecolors="black", linewidths=1.1, zorder=6)

    # guideline contours
    try:
        cs = ax.contour(grid_x, grid_y, zmask, levels=GUIDE_LEVELS,
                        colors="black", linewidths=0.8, linestyles="--")
        ax.clabel(cs, fmt=lambda v: f"{int(v)} µg/m³", inline=True, fontsize=8)
    except Exception:
        pass

    ax.set_title(f"PM₂.₅ Kriging — {day} (continuous, 0–100 µg/m³)")
    ax.set_axis_off()

    # road legend tucked inside
    leg_roads = ax.legend(handles=ROAD_HANDLES, title="Roads",
                          loc="lower left", frameon=True, fontsize=8)
    ax.add_artist(leg_roads)

    cb = plt.colorbar(mpl.cm.ScalarMappable(norm=NORM_CONT, cmap=CMAP_CONT), ax=ax, shrink=0.8)
    cb.set_label("PM₂.₅ (µg/m³)")
    cb.set_ticks([0, 25, 35, 45, 55, 90, 100])

    outA = os.path.join(OUT_F_CONT, f"{base}_kriged_continuous.png")
    plt.savefig(outA, bbox_inches="tight"); plt.close(fig)

    # === (B) Categorical map + roads
    idx = np.digitize(zmask, BINS, right=True) - 1
    cat_mask = np.ma.masked_where(~np.isfinite(zmask), idx)

    fig2, ax2 = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    ax2.imshow(cat_mask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
               origin="lower", cmap=CMAP_CAT, norm=NORM_CAT)
    pasig_utm.boundary.plot(ax=ax2, color="black", linewidth=0.8)

    # roads overlay
    for klass, style in STYLE_MAP.items():
        sub = roads_clip.loc[roads_clip[road_col] == klass]
        if not sub.empty:
            sub.plot(ax=ax2, **style, zorder=4)

    ax2.set_title(f"PM₂.₅ Kriging — {day} (local guideline)")
    ax2.set_axis_off()

    # legends: roads + category patches
    leg_roads2 = ax2.legend(handles=ROAD_HANDLES, title="Roads",
                            loc="lower left", frameon=True, fontsize=8)
    ax2.add_artist(leg_roads2)

    patches = [mpatches.Patch(color=c, label=l) for c, l in zip(COLORS, LABELS)]
    ax2.legend(handles=patches, title="PM₂.₅ category",
               loc="upper left", bbox_to_anchor=(1.02, 1.0),
               fontsize=9, frameon=True)

    outB = os.path.join(OUT_F_CAT, f"{base}_kriged_categorical.png")
    plt.savefig(outB, bbox_inches="tight"); plt.close(fig2)

    print("Saved:", os.path.basename(outA), "|", os.path.basename(outB))

Saved: date_2025-02-06_kriged_continuous.png | date_2025-02-06_kriged_categorical.png
Saved: date_2025-02-07_kriged_continuous.png | date_2025-02-07_kriged_categorical.png
Saved: date_2025-02-08_kriged_continuous.png | date_2025-02-08_kriged_categorical.png
Saved: date_2025-02-09_kriged_continuous.png | date_2025-02-09_kriged_categorical.png
Saved: date_2025-02-10_kriged_continuous.png | date_2025-02-10_kriged_categorical.png
Saved: date_2025-02-11_kriged_continuous.png | date_2025-02-11_kriged_categorical.png
Saved: date_2025-02-12_kriged_continuous.png | date_2025-02-12_kriged_categorical.png
Saved: date_2025-02-13_kriged_continuous.png | date_2025-02-13_kriged_categorical.png
Saved: date_2025-02-14_kriged_continuous.png | date_2025-02-14_kriged_categorical.png
Saved: date_2025-02-15_kriged_continuous.png | date_2025-02-15_kriged_categorical.png
Saved: date_2025-02-16_kriged_continuous.png | date_2025-02-16_kriged_categorical.png
Saved: date_2025-02-17_kriged_continuous.png | date_20